# Advanced Applications of Simulation
In this chapter, students will be introduced to some basic and advanced applications of simulation to solve real-world problems. We'll work through a business planning problem, learn about Monte Carlo Integration, Power Analysis with simulation and conclude with a financial portfolio simulation. After completing this chapter, students will be ready to apply simulation to solve everyday problems.

# 1. Simulation for Business Planning
## 1.1 Modeling Corn Production
Suppose that you manage a small corn farm and are interested in optimizing your costs. In this exercise, we will model the production of corn.

For simplicity, let's assume that corn production depends on only two factors: rain, which you don't control, and cost, which you control. Rain is normally distributed with mean 50 and standard deviation 15. For now, let's fix cost at 5,000. Corn produced in any season is a Poisson random variable while the average corn production is governed by the equation:

$100×(cost)^{0.1}×(rain)^{0.2}$
Let's model this production function and simulate one outcome.

### Instructions:
* Initialize `rain` as a normal random variable with mean 50 and standard deviation 15.
* In the `corn_produced()` function, model `mean_corn` as $100×cost^{0.1}×rain^{0.2}$.
* Model `corn` as a Poisson random variable with mean `mean_corn`.
* Simulate one outcome by storing the result of calling `corn_produced()` in `corn_result` and print your results.

In [1]:
import numpy as np
np.random.seed(223)

# Initialize variables
cost = 5000
rain = np.random.normal(50, 15)

# Corn Production Model
def corn_produced(rain, cost):
  mean_corn = 100*cost**0.1*rain**0.2
  corn = np.random.poisson(mean_corn)
  return corn

# Simulate and print corn production
corn_result = corn_produced(rain, cost)
print("Simulated Corn Production = {}".format(corn_result))

Simulated Corn Production = 560


## 1.2 Modeling Profits
In the previous exercise, you built a model of corn production. For a small farm, you typically have no control over the price or demand for corn. Suppose that price is normally distributed with mean 40 and standard deviation 10. You are given a function `corn_demanded()`, which takes the price and determines the demand for corn. This is reasonable because demand is usually determined by the market and is not in your control.

In this exercise, you will work on a function to calculate the profit by pulling together all the other simulated variables. The only input to this function will be the cost. Upon completion, you will have a function that will give you one simulated profit outcome for a given cost. This function can then be used for planning your costs.

### Instructions:
* Model the `price` as a normal random variable with mean 40 and standard deviation 10.
* Get the corn `supply` by calling the function `corn_produced(rain, cost)`, which you designed in the previous exercise.
* You are given a `corn_demanded()` function which takes `price` as an input. Call this function to get `demand`.
* Calculate the profit depending on the relationship between `supply` and `demand` of corn.

In [2]:
import numpy as np

# Corn production model
def corn_produced(rain, cost):
  mean_corn = 100*(cost**0.1)*(rain**0.2)
  corn = np.random.poisson(mean_corn)
  return corn

def corn_demanded(price):
    mean_corn = 1000 - 8*price
    corn = np.random.poisson(abs(mean_corn))
    return corn

np.random.seed(223)

cost = 5000

In [3]:
# Function to calculate profits
def profits(cost):
    rain = np.random.normal(50, 15)
    price = np.random.normal(40, 10)
    supply = corn_produced(rain, cost)
    demand = corn_demanded(price)
    equil_short = supply <= demand
    if equil_short ==True:
        tmp = supply * price - cost
        return tmp
    else: 
        tmp2 = demand * price - cost
        return tmp2
result = profits(cost)
print("Simulated profit = {}".format(result))

Simulated profit = 20675.3291075312


Now let's use these functions to optimize costs of production.

## 1.3 Optimizing Costs
Now we will use the functions you've built to optimize our cost of production. We are interested in maximizing average profits. However, our profits depend on a number of factors, but we only control cost. Thus, we can simulate the uncertainty in the other factors and vary cost to see how our profits are impacted.

Since you manage the small corn farm, you have the ability to choose your cost - from $100 to $5,000. You want to choose the cost that gives you the maximum average profit. In this exercise, we will simulate multiple outcomes for each cost level and calculate an average. We will then choose the cost that gives us the maximum mean profit. Upon completion, you will have a framework for selecting optimal inputs for business decisions.

### Instructions:
* Initialize the empty dictionary `results`.
* For each cost level, simulate profits using the pre-loaded `profits()` function and append them to `tmp_profits`.
* Store the average of `tmp_profits` for each cost level in the `results` dictionary.
* Find the cost level `cost_max` that has the maximum average profit by running `results` through the list comprehension.

In [4]:
import numpy as np
np.random.seed(223)

# Corn production model
def corn_produced(rain, cost):
  mean_corn = 100*(cost**0.1)*(rain**0.2)
  corn = np.random.poisson(mean_corn)
  return corn

# Corn demand
def corn_demanded(price):
    mean_corn = max(min(1000 - 10*price, 1000), 100)
    corn = np.random.poisson(abs(mean_corn))
    return corn

# Profits
def profits(cost):
    
    # Price is a normal random variable
    rain = max(np.random.normal(50, 15), 10)
    price = max(np.random.normal(40, 10), 10)
    
    # Call the appropriate functions for supply & demand
    supply, demand = corn_produced(rain, cost), corn_demanded(price)
    
    # Return the correct profits for each case
    if supply <= demand:
        return supply*price - cost
    else:
        return demand*price - cost

np.random.seed(573)

In [5]:
# Initialize results and cost_levels variables
sims, results = 1000, {}
cost_levels = np.arange(100, 5100, 100)

# For each cost level, simulate profits and store mean profit
for cost in cost_levels:
    tmp_profits = []
    for i in range(sims):
        tmp_profits.append(profits(cost))
    results[cost] = np.mean(tmp_profits)
    
# Get the cost that maximizes average profit
cost_max = [x for x in results.keys() if results[x] == max(results.values())][0]
print("Average profit is maximized when cost = {}".format(cost_max))

Average profit is maximized when cost = 1400


Businesses use a similar framework with more details to help in a number of decisions.

# 2. Monte Carlo Integration
## 2.1 Integrating a Simple Function
This is a simple exercise introducing the concept of Monte Carlo Integration.

Here we will evaluate a simple integral $\int_{0}^1xe^xdx$. We know that the exact answer is $1$, but simulation will give us an approximate solution, so we can expect an answer close to $1$. As we saw in the video, it's a simple process. For a function of a single variable $f(x)$:

1. Get the limits of the x-axis $(x_{min},x_{max})$ and y-axis $(max(f(x)),min(min(f(x)),0))$.
2. Generate a number of uniformly distributed point in this box.
3. Multiply the area of the box $((max(f(x)−min(f(x))×(xmax−xmin))$ by the fraction of points that lie below $f(x)$.

Upon completion, you will have a framework for handling definite integrals using Monte Carlo Integration.

### Instructions:
* In the `sim_integrate()` function, generate uniform random numbers between `xmin` and `xmax` and assign to `x`.
* Generate uniform random numbers between $min(min(f(x)),0)$ and $max(f(x))$ and assign to `y`.
* Return the fraction of points less than $f(x)$ multiplied by area $((max(f(x)−min(f(x))×(x_{max}−x_{min}))$.
* Finally, use lambda function to define `func` as $xe^x$.

In [17]:
np.random.seed(123)

# Define the sim_integrate function
def sim_integrate(func, xmin, xmax, sims):
    x = np.random.uniform(xmin, xmax, sims)
    y = np.random.uniform(min(min(func(x)), 0), max(func(x)), sims)
    area = (max(y) - min(y))*(xmax-xmin)
    result = area * sum(abs(y) < abs(func(x)))/sims
    return result
    
# Call the sim_integrate function and print results
result = sim_integrate(func = lambda x: x*np.exp(x), xmin = 0, xmax = 1, sims = 50)
print("Simulated answer = {}, Actual Answer = 1".format(result))

Simulated answer = 0.7240166789450252, Actual Answer = 1


Try seeing what happens to the answer when you increase or decrease `sims`.

In [16]:
np.random.seed(123)

# Call the sim_integrate function and print results
result = sim_integrate(func = lambda x: x*np.exp(x), xmin = 0, xmax = 1, sims = 10)
print("Simulated answer = {}, Actual Answer = 1".format(result))

Simulated answer = 0.8869683341751766, Actual Answer = 1


In [23]:
np.random.seed(123)

# Call the sim_integrate function and print results
result = sim_integrate(func = lambda x: x*np.exp(x), xmin = 0, xmax = 1, sims = 53)
print("Simulated answer = {}, Actual Answer = 1".format(result))

Simulated answer = 0.6342464168655881, Actual Answer = 1


In [27]:
np.random.seed(123)

# Call the sim_integrate function and print results
result = sim_integrate(func = lambda x: x*np.exp(x), xmin = 0, xmax = 1, sims = 52)
print("Simulated answer = {}, Actual Answer = 1".format(result))

Simulated answer = 0.9448019848870521, Actual Answer = 1


## 2.2 Calculating the value of pi
Now we work through a classic example - estimating the value of π.

Imagine a square of side $2$ with the origin $(0,0)$ as its center and the four corners having coordinates $(1,1),(1,−1),(−1,1),(−1,−1)$. The area of this square is $2×2=4$. Now imagine a circle of radius 1 with its center at the origin fitting perfectly inside this square. The area of the circle will be $\pi×radius^2=\pi$.

To estimate π, we randomly sample multiple points in this square & get the fraction of points inside the circle $(x^2+y^2<=1)$. The area of the circle then is $4$ times this fraction, which gives us our estimate of $\pi$.

After this exercise, you'll have a grasp of how to use simulation for computation.

### Instructions:
* Examine the true value of $\pi$ using `np.pi` in the console. Initialize `sims` to 10000 and circle_points to 0.
* Within the `for` loop, generate a point (x & y coordinates) using `np.random.uniform()` between -1 and 1, having `size=2`.
* Check if the point lies within the unit circle with the equation $x^2+y^2<=1$, assign to `within_circle`, and increment `circle_points` accordingly.
* Print the estimate of $\pi$ `pi_sim` as 4 times the fraction of points that lie within the circle.

In [28]:
np.random.seed(123)

# Initialize sims and circle_points
sims, circle_points = 10000, 0 

for i in range(sims):
    # Generate the two coordinates of a point
    point = np.random.uniform(-1, 1, 2)
    # if the point lies within the unit circle, increment counter
    within_circle = point[0]**2 + point[1]**2 <= 1
    if within_circle == True:
        circle_points +=1
        
# Estimate pi as 4 times the avg number of points in the circle.
pi_sim = 4*np.mean(circle_points/sims)
print("Simulated value of pi = {}".format(pi_sim))

Simulated value of pi = 3.1468


Similar to this example, simulations have a number of useful applications in computation.